# Solow-model with human capital and technological growth 

Imports and set magics:

In [45]:
# Imports required to read the code
import numpy as np
from scipy import optimize
import sympy as sm
from sympy import *

# Imports required to plot
import ipywidgets as widgets
import matplotlib.pyplot as plt

# Autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# Local modules
from modelproject import SolowClass
model = SolowClass()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
initializing the model:
calling .setup()


# Description of the Solow model with human capital

The Solow model with human capital contains the following variables:
1. $K_t$ is physical capital
2. $H_t$ is human capital
3. $L_t$ is labor (growing with a constant rate of $n$)
4. $A_t$ is technology (growing with a constant rate of $g$)
5. $Y_t = F(K_t,H_t,A_tL_t)$ is GDP
6. $s_K$ is the savings rate for physical capital
7. $s_H$ is the savings rate for human capital

The Solow model with human capital is defined by the following equations:

The Cobb-Douglas production function:
$$ Y_{t} = F(K_{t},H_{t},A_{t}L_{t})=K_{t}^{\alpha}H_{t}^{\phi}(A_{t}L_{t})^{1-\alpha-\phi} $$
  where
  $$ 0<\alpha<1$$ 
  $$0<\phi<1$$
  $$\alpha+\phi<1  $$
with growth in labor and technology given by:
$$ L_{t+1}=(1+n)L_{t}$$
$$ A_{t+1}=(1+g)A_{t}$$


Saving of human and physical capital as a constant fraction of GDP:
$$ S_t = (s_{K}+s_{H})Y_t $$
  where
  $$ s_{K}+s_{H} <1 $$

Capital accumulation for physical capital:
$$ K_{t+1} = s_{K}Y_{t}+(1-\delta)K_{t}$$

Capital accumulation for human capital:
$$ H_{t+1} = s_{H}Y_{t}+(1-\delta)H_{t}$$

Technological adjusted variables are given by:
$$\tilde{k}_t = \frac{K_t}{A_{t}L_{t}}$$
$$\tilde{h}_t = \frac{H_t}{A_{t}L_{t}}$$
$$\tilde{y}_t = \frac{Y_t}{A_{t}L_{t}} = \frac{K_{t}^{\alpha}H_{t}^{\phi}(A_{t}L_{t})^{1-\alpha-\phi}}{A_{t}L_{t}} = \tilde{k}_t^\alpha \tilde{h}_t^\phi$$
which implies that the productions function has constant return to scale.

The transition equation for physical capital:
$$\tilde{k}_{t+1}=\frac{1}{(1+n)(1+g)}[s_{K}\tilde{k}_t^\alpha \tilde{h}_t^\phi+(1-\delta)\tilde{k}_t]$$

The transition equation for human capital:
$$\tilde{h}_{t+1}=\frac{1}{(1+n)(1+g)}[s_{H}\tilde{k}_t^\alpha \tilde{h}_t^\phi+(1-\delta)\tilde{h}_t]$$

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

In [36]:
k = sm.symbols('k')
h = sm.symbols('h')
alpha = sm.symbols('alpha')
phi = sm.symbols('phi')
delta = sm.symbols('delta')
s_K = sm.symbols('s_K')
s_H = sm.symbols('s_H')
g = sm.symbols('g')
n = sm.symbols('n')

In [37]:
# Define f(k*h*)
f = k**alpha*h**phi

# Define steady state for physical capital 
ss_K = sm.Eq(k,(s_K*f+(1-delta)*k)/((1+n)*(1+g)))

# Solve steady state for k
kss = sm.solve(ss_K,k)[0]

# Display solution 
kss

((delta + g*n + g + n)/(h**phi*s_K))**(1/(alpha - 1))

In [38]:
# Define steady state for human capital
ss_H = sm.Eq(h,(s_H*f+(1-delta)*h)/((1+n)*(1+g)))

# Solve steady state for h
hss = sm.solve(ss_H,h)[0]

# Display solution 
hss

((delta + g*n + g + n)/(k**alpha*s_H))**(1/(phi - 1))

In [39]:
# kss-function with 5 arguments 
ss_func_k = sm.lambdify((s_K, s_H,g,n,delta,alpha, phi),kss)

# Evaluate function
ss_func_k(0.2,0.2,0.02,0.01,0.1,1/3, 1/3)

1.90383153923132*h**0.5

In [40]:
# kss-function with 5 arguments 
ss_func_h = sm.lambdify((s_K, s_H,g,n,delta,alpha, phi),hss)

# Evaluate function
ss_func_h(0.2,0.2,0.02,0.01,0.1,1/3, 1/3)

1.90383153923132*k**0.5

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

### Algorithm

The algorithm contains XX steps:
1. Initialize the parameters of the model given by, the saving rate in physical and human capital, s_K and s_H, the growth in technology, g, the growth in population, n, return on physical and human capital, alpha and phi, and the depriciation rate, delta.

2. Set the function for the Solow equations, given by an equation for physical capital and an equation for human capital.
    - Set the variables and parameters contained in the equations
    - Define the Solow equations for $\tilde{k}$ and $\tilde{h}$

3. Set up an optimizer to solve the equations
    - Set initial guess for the values of $\tilde{k}$ and $\tilde{h}$
    - Call the optimize.root solver


In [51]:
# Step 1: Initialize the baseline parameters 
s_K = 0.2
s_H = 0.2
g = 0.02
n = 0.01
alpha = 1/3
phi = 1/3
delta = 0.1

In [54]:
# Step 2: Set the Solow equations
model.solow_equations

NameError: name 'solow_k' is not defined

In [49]:
# Step 3: Set up the solver

# Initial guess for k_tilde and h_tilde
vars0 = [1 , 1]

model.solow_equations

# Solver
solution = optimize.root(fun = model.solow_equations, vars0, method='hybr', args = None)

SyntaxError: positional argument follows keyword argument (2406840778.py, line 9)

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

## Extenting the Solow model with human capital to include R&D

I extend the Solow model with human capital by adding R&D, shown by having to different types of labor. This implies that the production function and the capital accumulation equations for both physical and human capital are unchanged. 

The Solow model with human capital and R&D contains the following variables:
1. $K_t$ is phisical capital
2. $H_t$ is human capital
3. $L_{t}$ is labor (growing with a constant rate of $n$)
4. $L_{Yt}$ is production labor
5. $L_{At}$ is research labor
6. $A_t$ is technology (growing with a constant rate of $g$)
7. $Y_t = F(K_t,A_tL_t)$ is GDP
8. $s_K$ is the savings rate for physical capital
9. $s_H$ is the savings rate for human capital
10. $s_R$ is the share of the total population who are research labor 

The Solow model with human capital is defined by the following equations:

The Cobb-Douglas production function:
$$ Y_{t} = F(K_{t},H_{t},A_{t}L_{t})=K_{t}^{\alpha}H_{t}^{\phi}(A_{t}L_{t})^{1-\alpha-\phi} $$
  where
  $$ 0<\alpha<1$$ 
  $$0<\phi<1$$
  $$\alpha+\phi<1  $$
with growth in the total labor force and technology given by:
$$ L_{t+1}=(1+n)L_{t}$$
$$ A_{t+1}=\rho A_{t}^{\sigma} L_{At}^{\lambda}+A_{t}$$
where
$$ \rho >0 $$
$$ \sigma >0 $$
$$ 0=<\lambda<1 $$

The share of research labor is given by:
$$ L_{At}=s_{R}L_{t}$$
where
$$ 0<s_{R}<1$$

The total labor force is given by:
$$ L_{t}=L_{At}+L_{Yt} $$

Saving of human and physical capital as a constant fraction of GDP:
$$ S_t = (s_{K}+s_{H})Y_t $$
  where
  $$ s_{K}+s_{H} <1 $$

Capital accumulation for physical capital:
$$ K_{t+1} = s_{K}Y_{t}+(1-\delta)K_{t}$$

Capital accumulation for human capital:
$$ H_{t+1} = s_{H}Y_{t}+(1-\delta)H_{t}$$

Technological adjusted variables are given by:
$$\tilde{k}_t = \frac{K_t}{L_{Yt}}$$
$$\tilde{h}_t = \frac{H_t}{L_{Yt}}$$
$$\tilde{y}_t = \frac{Y_t}{L_{Yt}} = \frac{K_{t}^{\alpha}H_{t}^{\phi}(A_{t}L_{Yt})^{1-\alpha-\phi}}{L_{Yt}} = \tilde{k}_t^\alpha \tilde{h}_t^\phi A_t^{1-\alpha-\phi}$$

The transition equation for physical capital:
$$\tilde{k}_{t+1}=\frac{1}{(1+n)(1+g)}[s_{K}\tilde{k}_t^\alpha \tilde{h}_t^\phi+(1-\delta)\tilde{k}_t]$$

The transition equation for human capital:
$$\tilde{h}_{t+1}=\frac{1}{(1+n)(1+g)}[s_{H}\tilde{k}_t^\alpha \tilde{h}_t^\phi+(1-\delta)\tilde{h}_t]$$

# Conclusion

Add concise conclusion. 